In [26]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tcn import TCN
import warnings

warnings.filterwarnings('ignore')


In [12]:
# df['ML_Node'].value_counts()

In [13]:
selected_features = ['timestamp_seconds', # FixMe
                     'node_memory_Percpu_bytes', 
                     'node_context_switches_total', 
                     'surfsara_power_usage', 
                     'node_netstat_Tcp_InSegs', 
                     'node_netstat_Tcp_OutSegs', 
                     'node_network_transmit_packets_total-sum', 
                     'node_filesystem_size_bytes-sum', 
                     'node_filesystem_files-sum', 
                     'node_memory_MemFree_bytes', 
                     'node_netstat_Tcp_InErrs']
# FixMe

In [14]:
file_path = "myData2.parquet"
df = pd.read_parquet(file_path)


In [15]:
# Define time intervals
time_intervals = {'minute': '1T', 'hour': '1H', 'day': '1D'}

# Set sequence length
sequence_length = 30

# Number of time steps to predict into the future
prediction_steps_tcn = 7


In [16]:
# Function to prepare data for TCN
def prepare_tcn_data(data, time_interval):
    data.set_index('timestamp', inplace=True)
    data_resampled = data.resample(time_interval).sum()
    
    target_mean = data_resampled['target'].mean()
    target_std = data_resampled['target'].std()
    data_resampled['target'] = (data_resampled['target'] - target_mean) / target_std
    
    target_min = data_resampled['target'].min()
    target_max = data_resampled['target'].max()
    print("Minimum value of target variable:", target_min)
    print("Maximum value of target variable:", target_max)
    
    return data_resampled

# Function to create sequences for TCN
def create_tcn_sequences(data, sequence_length):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data.iloc[i:i+sequence_length].values
        target = data.iloc[i+sequence_length]['target']
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)

# Function to make predictions on new data for TCN model
def predict_future_failures_tcn(model, input_data, sequence_length, prediction_steps):
    predictions = []

    for _ in range(prediction_steps):
        # Make a prediction for the next time step
        prediction = model.predict(input_data.reshape(1, sequence_length, input_data.shape[1]))
        predictions.append(prediction[0, 0])

        # Shift the input data by one time step and append the new prediction
        input_data = np.roll(input_data, shift=-1, axis=0)
        input_data[-1, -1] = prediction[0, 0]

    return predictions


# Day

In [17]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])



In [18]:
# Prepare data for TCN with day intervals
tcn_data_day = prepare_tcn_data(df_selected, time_intervals['day'])

# Create sequences and targets
sequences_day, targets_day = create_tcn_sequences(tcn_data_day, sequence_length)

# Split the data into training and testing sets
X_train_day, X_test_day, y_train_day, y_test_day = train_test_split(sequences_day, targets_day, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_day = Sequential([
    TCN(input_shape=(sequence_length, X_train_day.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_day.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
history_day = tcn_model_day.fit(X_train_day, y_train_day, epochs=20, batch_size=32, validation_split=0.15)


Minimum value of target variable: -0.7362924419461765
Maximum value of target variable: 5.322951443655186
Epoch 1/20
2/2 [==============================] - 3s 456ms/step - loss: 1.6825 - mean_squared_error: 1.6825 - val_loss: 1.5427 - val_mean_squared_error: 1.5427
Epoch 2/20
2/2 [==============================] - 0s 86ms/step - loss: 1.6825 - mean_squared_error: 1.6825 - val_loss: 1.5427 - val_mean_squared_error: 1.5427
Epoch 3/20
2/2 [==============================] - 0s 87ms/step - loss: 1.6825 - mean_squared_error: 1.6825 - val_loss: 1.5427 - val_mean_squared_error: 1.5427
Epoch 4/20
2/2 [==============================] - 0s 84ms/step - loss: 1.6825 - mean_squared_error: 1.6825 - val_loss: 1.5427 - val_mean_squared_error: 1.5427
Epoch 5/20
2/2 [==============================] - 0s 82ms/step - loss: 1.6825 - mean_squared_error: 1.6825 - val_loss: 1.5427 - val_mean_squared_error: 1.5427
Epoch 6/20
2/2 [==============================] - 0s 83ms/step - loss: 1.6825 - mean_squared_error

## Performance_Day

In [ ]:
# Evaluate the model using Mean Squred Error
mse_day = tcn_model_day.evaluate(X_test_day, y_test_day, verbose=0)[1]
print(f'Model Mean Squared Error: {mse_day:.4f}\n')


In [ ]:
# Evaluate model on test data
y_pred_day = tcn_model_day.predict(X_test_day)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test_day, y_pred_day)
print("Mean Absolute Error (MAE):", mae)

# Calculate Root Mean Squared Error
rmse = mean_squared_error(y_test_day, y_pred_day, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared
r2 = r2_score(y_test_day, y_pred_day)
print("R-squared (R2):", r2)

# Additional: Compare with training and validation loss from history
train_loss = history_day.history['loss']
val_loss = history_day.history['val_loss']


### Training and Validation Loss Plot

In [ ]:
# Plot training and validation loss
plt.plot(history_day.history['loss'], label='Training Loss')
plt.plot(history_day.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


### Predicted Failures vs. True Failures Plot

In [ ]:
index_day = 14

input_data = X_test_day[index_day]  # Can be any valid starting point

# Make predictions
predicted_failures_day = predict_future_failures_tcn(tcn_model_day, input_data, sequence_length, prediction_steps_tcn)#prediction_steps

# Denormalize the predicted failures 
predicted_failures_denormalized_day = np.array(predicted_failures_day) * (tcn_data_day['target'].max() - tcn_data_day['target'].min()) + tcn_data_day['target'].min()

# Get the true failures for the specified number of hours
true_failures_day = y_test_day[index_day:index_day + prediction_steps_tcn]  # Adjust the range as needed #prediction_steps

# Print the predicted failures
print("Predicted failures for the next 7 days:")
print(predicted_failures_denormalized_day)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_day - true_failures_day)**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_day - true_failures_day))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}')

# Plot predicted failures vs. true failures
plt.figure(figsize=(10, 6))
plt.plot(predicted_failures_denormalized_day, label='Predicted Failures', marker='o')
plt.plot(true_failures_day, label='True Failures', marker='x')
plt.xlabel('Hours')
plt.ylabel('Number of Failures')
plt.title('Predicted Failures vs. True Failures')
plt.legend()
plt.grid(True)
plt.show()


In [19]:


# Select a starting point for predictions
input_data_tcn = X_test_day[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_day, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_day[3:3 + prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_day[3:3 + prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 1.9790

1/1 [==============================] - 0s 22ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Mean Squared Error for Predictions: 3.9855

Mean Absolute Error for Predictions: 1.6349



# Hour

In [20]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])



In [21]:
# Prepare data for TCN
tcn_data_hour = prepare_tcn_data(df_selected, time_intervals['hour'])

# Create sequences and targets
sequences_hour, targets_hour = create_tcn_sequences(tcn_data_hour, sequence_length)

# Split the data into training and testing sets
X_train_hour, X_test_hour, y_train_hour, y_test_hour = train_test_split(sequences_hour, targets_hour, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_hour = Sequential([
    TCN(input_shape=(sequence_length, X_train_hour.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_hour.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_hour.fit(X_train_hour, y_train_hour, epochs=20, batch_size=32, validation_split=0.15)


Minimum value of target variable: -0.44004356707964004
Maximum value of target variable: 22.33986610637043
Epoch 1/20
51/51 [==============================] - 5s 53ms/step - loss: 0.6922 - mean_squared_error: 0.6922 - val_loss: 0.5196 - val_mean_squared_error: 0.5196
Epoch 2/20
51/51 [==============================] - 2s 46ms/step - loss: 0.6549 - mean_squared_error: 0.6549 - val_loss: 0.5196 - val_mean_squared_error: 0.5196
Epoch 3/20
51/51 [==============================] - 2s 43ms/step - loss: 0.6549 - mean_squared_error: 0.6549 - val_loss: 0.5196 - val_mean_squared_error: 0.5196
Epoch 4/20
51/51 [==============================] - 2s 43ms/step - loss: 0.6549 - mean_squared_error: 0.6549 - val_loss: 0.5196 - val_mean_squared_error: 0.5196
Epoch 5/20
51/51 [==============================] - 2s 45ms/step - loss: 0.6549 - mean_squared_error: 0.6549 - val_loss: 0.5196 - val_mean_squared_error: 0.5196
Epoch 6/20
51/51 [==============================] - 2s 44ms/step - loss: 0.6549 - mean_s

In [22]:
# Evaluate the model using Mean Squred Error
mae_hour = tcn_model_hour.evaluate(X_test_hour, y_test_hour, verbose=0)[1]
print(f'Model Mean Squared Error: {mae_hour:.4f}\n')

# Select a starting point for predictions
input_data_tcn = X_test_hour[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_hour, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_hour[3:3+prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_hour[3:3+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 1.8850

1/1 [==============================] - 0s 22ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Squared Error for Predictions: 0.1671

Mean Absolute Error for Predictions: 0.4041



# Minute

In [23]:
# Extract relevant columns
df_selected = df[['timestamp', 'state'] + selected_features].copy()

# Encode the target variable 'state' to binary (0 for "COMPLETED", 1 otherwise)
df_selected['target'] = (df_selected['state'] != 'COMPLETED').astype(int)

# Drop the original 'state' column
df_selected.drop('state', axis=1, inplace=True)

# Normalize selected features
scaler = MinMaxScaler()
df_selected[selected_features] = scaler.fit_transform(df_selected[selected_features])



In [24]:
# Prepare data for TCN
tcn_data_minute = prepare_tcn_data(df_selected, time_intervals['minute'])

# Create sequences and targets
sequences_minute, targets_minute = create_tcn_sequences(tcn_data_minute, sequence_length)

# Split the data into training and testing sets
X_train_minute, X_test_minute, y_train_minute, y_test_minute = train_test_split(sequences_minute, targets_minute, test_size=0.3, random_state=1)

# Build the TCN model
tcn_model_minute = Sequential([
    TCN(input_shape=(sequence_length, X_train_minute.shape[2])),
    Dense(1, activation='sigmoid')
])

# Compile the model
tcn_model_minute.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

# Train the model
tcn_model_minute.fit(X_train_minute, y_train_minute, epochs=10, batch_size=64, validation_split=0.15)


Minimum value of target variable: -0.43499466427008204
Maximum value of target variable: 22.077561101301857
Epoch 1/10
1539/1539 [==============================] - 96s 61ms/step - loss: 0.9966 - mean_squared_error: 0.9966 - val_loss: 0.9588 - val_mean_squared_error: 0.9588
Epoch 2/10
1539/1539 [==============================] - 93s 60ms/step - loss: 0.9966 - mean_squared_error: 0.9966 - val_loss: 0.9588 - val_mean_squared_error: 0.9588
Epoch 3/10
1539/1539 [==============================] - 90s 59ms/step - loss: 0.9966 - mean_squared_error: 0.9966 - val_loss: 0.9588 - val_mean_squared_error: 0.9588
Epoch 4/10
1539/1539 [==============================] - 92s 60ms/step - loss: 0.9966 - mean_squared_error: 0.9966 - val_loss: 0.9588 - val_mean_squared_error: 0.9588
Epoch 5/10
1539/1539 [==============================] - 91s 59ms/step - loss: 0.9966 - mean_squared_error: 0.9966 - val_loss: 0.9588 - val_mean_squared_error: 0.9588
Epoch 6/10
1539/1539 [==============================] - 90s 58

In [25]:
# Evaluate the model using Mean Squred Error
mae_minute = tcn_model_minute.evaluate(X_test_minute, y_test_minute, verbose=0)[1]
print(f'Model Mean Squared Error: {mae_hour:.4f}\n')

# Select a starting point for predictions
input_data_tcn = X_test_minute[3]

# Make predictions with the TCN model
predicted_failures_tcn = predict_future_failures_tcn(tcn_model_minute, input_data_tcn, sequence_length, prediction_steps_tcn)

# Print the predicted failures for TCN
print("Predicted Failures for the Next 7 Time Steps (TCN):")
print(predicted_failures_tcn)

# Evaluate the predictions using Mean Squared Error
mse_predictions = np.mean((predicted_failures_tcn - y_test_minute[3:3+prediction_steps_tcn])**2)
print(f'\nMean Squared Error for Predictions: {mse_predictions:.4f}\n')

# Evaluate the predictions using Mean Absolute Error
mae_predictions = np.mean(np.abs(predicted_failures_tcn - y_test_minute[3:3+prediction_steps_tcn]))
print(f'Mean Absolute Error for Predictions: {mae_predictions:.4f}\n')


Model Mean Squared Error: 1.8850

1/1 [==============================] - 0s 24ms/step
Predicted Failures for the Next 7 Time Steps (TCN):
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Squared Error for Predictions: 0.3884

Mean Absolute Error for Predictions: 0.5647

